---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [11]:
def date_sorter():
    # %%
    # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    fmt1_a = df.str.extractall(r'(\d{1,2})[-/](\d{1,2})[-/](\d{2,4})')
    #fmt1_b = df.str.extractall(r'(\d{1,2})[-/](\d{1,2})[-/](\d{4})')
    fmt1 = pd.concat([fmt1_a])
    fmt1.reset_index(inplace=True)
    fmt1_index = fmt1['level_0']

    # %%
    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    fmt2 = df.str.extractall(
            r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]* )'
            '(\d{1,2})[?:, -]*(\d{4})')

    #fmt2_b = df.str.extractall(
    #        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d{1,2}),\s'
    #        '(\d{2,4})')
    #
    #fmt2_c = df.str.extractall(
    #        r'(January|February|March|April|May|June|July|August|'
    #        'September|October|November|December)\s(\d{1,2}),\s(\d{2,4})')
    #
    #fmt2_d = df.str.extractall(
    #        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\.\s(\d{1,2}),\s'
    #        '(\d{2,4})')
    #
    #fmt2_e = df.str.extractall(
    #        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d{1,2})\s'
    #        '(\d{2,4})')

    #fmt2 = pd.concat([fmt2_a, fmt2_b, fmt2_c, fmt2_d, fmt2_e])

    fmt2.reset_index(inplace=True)
    fmt2_index = fmt2['level_0']

    # %%
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009

    fmt3 = df.str.extractall(r'((?:\d{1,2} ))?'
               '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)'
               '[a-z]*[?:, -]* )(\d{4})')

    fmt3.rename(columns={0:1, 1:0}, inplace=True)
    fmt3.reset_index(inplace=True)
    fmt3_index = fmt3['level_0']

    # %%
    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    #fmt4 = df.str.extractall(
    #        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d{1,2})'
    #        '(st|nd|rd|th),\s(\d{2,4})')
    #fmt4.index.rename('level_0', inplace=True)
    #fmt4.reset_index(inplace=True)
    #fmt4_index = fmt4['level_0']


    # %%
    # Feb 2009; Sep 2009; Oct 2010
    #fmt5 = df.str.extractall(
    #        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d{2,4})')
    #
    #fmt5 = fmt5.rename(columns={1:2})
    #fmt5.reset_index(inplace=True)
    #fmt5_index = fmt5['level_0']

    # %%
    # 6/2008; 12/2009
    fmt6 = df.str.extractall(r'(\d{1,2})/(\d{4})')

    fmt6 = fmt6.rename(columns={1:2})
    fmt6.reset_index(inplace=True)
    fmt6_index = fmt6['level_0']

    # %%
    # 2009; 2010
    fmt7_a = df.str.extractall(r'.*(\d{4})')
    fmt7_b = df.str.extractall(r'^(\d{4})[^0-9]')

    fmt7 = pd.concat([fmt7_a, fmt7_b])

    fmt7 = fmt7.rename(columns={0:2})
    fmt7.reset_index(inplace=True)
    fmt7_index = fmt7['level_0']

    # %%
    df2 = pd.concat([fmt1, fmt2, fmt3, fmt6, fmt7])
    df2['level_0'] = df2['level_0'].astype(int)

    for col in df2.select_dtypes(include=['object']):
        df2[col] = df2[col].str.strip()

    df2.rename(columns={0:'month', 1:'day', 2:'year'}, inplace=True)
    df2 = df2[df2['match'] == 0]
    df2.drop('match', axis=1, inplace=True)

    months = ['January', 'February', 'March', 'April', 'May', 'June',
              'July', 'August', 'September', 'October', 'November', 'December']

    months_num = dict(enumerate(months, start=1))
    months_dict = {v:k for k,v in months_num.items()}
    months_dict3 = {v[:3]:k for k,v in months_num.items()}

    df2['month'] = df2['month'].str.replace(r'[\,\.]', '')
    df2['month'] = df2['month'].replace({'Decemeber':'December',
                                         'Janaury':'January'})
    df2['month'].replace(months_dict, inplace=True)
    df2['month'].replace(months_dict3, inplace=True)
    df2['month'].fillna(1, inplace=True)
    df2['day'].fillna(1, inplace=True)
    df2['month'] = df2['month'].astype(int)
    df2['month'] = np.where(df2['month'] > 12, df2['day'], df2['month'])
    df2['month'] = np.where(df2['month'] > 12, 1, df2['month'])

    df2['day'] = df2['day'].astype(int)
    df2['year'] = df2['year'].astype(int)
    df2['year'] = np.where(df2['year'].astype(str).str.len() == 2,
                           '19' + df2['year'].astype(str),
                           df2['year'])

    df2 = df2.drop_duplicates(subset='level_0', keep='first')

    df2['date'] = (
            pd.to_datetime(df2['month'].astype(str)
            + '/'
            + df2['day'].astype(str)
            + '/'
            + df2['year'].astype(str)))


    df2 = df2.sort_values(by='date')
    ordered = list(enumerate(df2['level_0']))

    chrono = pd.Series(data=[i[1] for i in ordered],
                       index=[i[0] for i in ordered])


    return chrono
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     415
17     335
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    480
477    244
478    286
479    431
480    279
481    198
482    463
483    381
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64